# Student: Gabriel Greenstein
## SUNet ID: gbg222

# Import Packages

In [15]:
from openmm import *
from openmm.app import *
from openmm.unit import *
from sys import stdout
import openmmtools
import numpy as np

# Defining our Integrators
### Note: 
The following methods are free of any distance constraints as that requires separate algorithms [1].

## Euler's Method

The context stores the complete state of a simulation, which includes the current time, position of each particle, velocity of each particle, and the values of configurable parameters defined by Force objects in the System [2]. Since the integrator is called at every time step, it is important that we first make sure the state of the simulation (i.e., the context) is up to date. Then we proceed to calculate the new positions and velocities of each particle using Euler's method:

$$
\begin{align*}
    \begin{split}
        x(t+\Delta t) &= x(t) + v(t)\cdot\Delta t \\ \\
        v(t+\Delta t) &= v(t) +  \frac{F(x(t))}{m}\cdot\Delta t
    \end{split}
\end{align*}
$$

In [32]:
class Euler(openmm.CustomIntegrator):
    def __init__(self, timestep):
        super(Euler, self).__init__(timestep)
        self.addUpdateContextState()
        self.addComputePerDof("x", "x+dt*v")   # x = x + v*dt
        self.addComputePerDof("v", "v+dt*f/m") # v = v + dt*(f/m)

## Velocity Verlet Method

Again, we first make sure the state of the simulation is up to date by updating the context. We then proceed to calculate the new positions and velocities of each particle using

$$
\begin{align*}
    \begin{split}
        v\left(\frac{\Delta t}{2}\right) &= v(0) + \frac{\Delta t}{2m}\cdot F(x(0))\\\\
        x(\Delta t) &= x(0) + v\left(\frac{\Delta t}{2}\right)\cdot\Delta t\\\\
        v(\Delta t) &= v\left(\frac{\Delta t}{2}\right) + \frac{\Delta t}{2m}\cdot F(x(\Delta t))
    \end{split}
\end{align*}
$$

Note that we don't update $F(x(t))$ between lines (1) and (3). While it looks like the forces we use are the same – which would be wrong – they are actually appropriately updated automatically. OpenMM recognizes that the positions have changed and recomputes the forces automatically [1].

In [33]:
class VelocityVerlet(openmm.CustomIntegrator):
    def __init__(self, timestep):
        super(VelocityVerlet, self).__init__(timestep)
        self.addUpdateContextState()
        self.addComputePerDof("v", "v+0.5*dt*f/m") # v = v + 0.5*dt(f/m)
        self.addComputePerDof("x", "x+dt*v")       # x = x + dt*v
        self.addComputePerDof("v", "v+0.5*dt*f/m") # v = v + 0.05*dt*(f/m)

# Define Helper Functions
## Reverse Trajectory:


Assume a particle starts from any point $(x_0,v_0)$. Over time $t_f$, our particle moves to the point $(x_f,v_f)$ by Newton's equations. Now, invert the velocity at this point, i.e. $(x_f,-v_f)$. If we now start at this point and follow our particle over time $t,$ we will eventually arrive at $(x_0,-v_0)$. Since only the positions are used to create the PDB file, this is exactly what is desirable [3].

In [34]:
def reverse_velocities(velocities):
    
    rv = []

    for i in range(len(velocities)):
        newVelocity = velocities[i].__mul__(-1)
        rv.append(newVelocity)
        
    return rv

## Compute Trajectory:

In [35]:
def compute(input_file, method, reverse, velocities = None):
    
    # Read in modified PDB
    pdb = PDBFile(input_file)

    # Initialize AMBER forcefield
    forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

    # Generate our system
    system = forcefield.createSystem(pdb.topology)

    # Specify our parameters: temperature, time step distance, total # of time steps
    temperature = 300.0*kelvin
    timestep = 0.01 * femtoseconds
    total_steps = 1000000

    # Define our integrator
    if method == "E":
        integrator = Euler(timestep)
    if method == "VV":
        integrator = VelocityVerlet(timestep)

    # Run our simulation: set positions, minimize energy
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()

    # Set initial velocities
    if reverse == False:
        simulation.context.setVelocitiesToTemperature(temperature)
    if reverse == True and velocities != None:
        simulation.context.setVelocities(reverse_velocities(velocities))
        
    # Define how often we report
    reportInterval = 1000

    # Add frame to PDB file and console out simulation info
    if reverse == False:
        simulation.reporters.append(PDBReporter(str(method)+ '_forward_sim.pdb', 5000))
    if reverse == True:
        simulation.reporters.append(PDBReporter(str(method)+ '_reverse_sim.pdb', 5000))

    simulation.reporters.append(StateDataReporter(stdout, 1000, progress=True, remainingTime = True, step=True,
                                                  potentialEnergy=True, temperature=True, totalSteps = total_steps))

    # Run simulation for total_steps
    simulation.step(total_steps)

    # Once done: get the positions and velocities of final state
    positions = simulation.context.getState(getPositions=True).getPositions()
    velocities = simulation.context.getState(getVelocities=True).getVelocities()

    # Write positions to PDB file
    if reverse == False:
        PDBFile.writeFile(simulation.topology, positions, open(str(method)+ '_forward_final.pdb', 'w'))
    if reverse == True:
        PDBFile.writeFile(simulation.topology, positions, open(str(method)+ '_reverse_final.pdb', 'w'))

    return velocities

# Performing Simulations
## Euler Method (Forward in time)

In [36]:
E_velocities = compute('data/2f4k-processed.pdb','E', False)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Time Remaining"
0.1%,1000,-2884.405029296875,169.49695184098937,--
0.2%,2000,-2709.39892578125,144.98597833902028,2:21
0.3%,3000,-2656.94140625,137.5643913501841,2:29
0.4%,4000,-2719.565673828125,146.27888572903882,2:30
0.5%,5000,-2820.069580078125,160.37779620756982,2:31
0.6%,6000,-2750.51318359375,150.71144357386672,2:32
0.7%,7000,-2707.908447265625,144.73513857680481,2:31
0.8%,8000,-2756.12939453125,151.43752731825467,2:31
0.9%,9000,-2711.745361328125,145.26177270841598,2:30
1.0%,10000,-2755.188720703125,151.33529173113018,2:31
1.1%,11000,-2682.658203125,141.1971846749217,2:30
1.2%,12000,-2720.4521484375,146.43560951369668,2:30
1.3%,13000,-2799.400390625,157.45584320925983,2:29
1.4%,14000,-2752.45458984375,150.9345370811683,2:30
1.5%,15000,-2788.546875,155.97844875742723,2:30
1.6%,16000,-2675.39404296875,140.2101329341626,2:30
1.7%,17000,-2615.9892578125,131.87980503016942,2:30
1.8%,18000,-2676.8330078125,140.358

15.5%,155000,-2770.90771484375,153.49718458946444,2:08
15.6%,156000,-2737.97119140625,148.8748627877162,2:08
15.7%,157000,-2777.020751953125,154.3320064175272,2:08
15.8%,158000,-2758.1123046875,151.68436685353467,2:08
15.9%,159000,-2744.32080078125,149.75826609524177,2:07
16.0%,160000,-2744.38134765625,149.77069233210503,2:07
16.1%,161000,-2777.19580078125,154.34637319001024,2:07
16.2%,162000,-2781.111572265625,154.89705974996087,2:07
16.3%,163000,-2763.04541015625,152.39867120626593,2:07
16.4%,164000,-2698.59814453125,143.39673073317587,2:07
16.5%,165000,-2746.307373046875,150.03016237115568,2:06
16.6%,166000,-2787.481201171875,155.77086251284135,2:06
16.7%,167000,-2762.555908203125,152.30678514244678,2:06
16.8%,168000,-2740.3076171875,149.21077971683945,2:06
16.9%,169000,-2753.2470703125,151.0131457960102,2:06
17.0%,170000,-2746.7802734375,150.1145926928569,2:06
17.1%,171000,-2750.859375,150.67261583922954,2:06
17.2%,172000,-2731.324951171875,147.93678403799225,2:05
17.3%,173000,-279

30.6%,306000,-2702.984130859375,143.99659455106902,1:45
30.7%,307000,-2762.03564453125,152.2216398865699,1:45
30.8%,308000,-2750.08984375,150.55533258993088,1:45
30.9%,309000,-2767.23974609375,152.94195309082542,1:45
31.0%,310000,-2740.4033203125,149.20060041595696,1:45
31.1%,311000,-2766.7314453125,152.86215281902398,1:45
31.2%,312000,-2742.869873046875,149.5572163916908,1:44
31.3%,313000,-2754.3974609375,151.18410017521285,1:44
31.4%,314000,-2745.420654296875,149.9148366797191,1:44
31.5%,315000,-2721.9921875,146.63211427855927,1:44
31.6%,316000,-2814.275146484375,159.50525309061794,1:44
31.7%,317000,-2776.145263671875,154.20963351778732,1:44
31.8%,318000,-2725.8173828125,147.18193270415372,1:43
31.9%,319000,-2786.456298828125,155.63128012615792,1:43
32.0%,320000,-2730.13330078125,147.76375294523166,1:43
32.1%,321000,-2763.674560546875,152.4517465576902,1:43
32.2%,322000,-2756.755859375,151.49911379081814,1:43
32.3%,323000,-2699.159423828125,143.4466910142632,1:43
32.4%,324000,-2731.8

45.7%,457000,-2804.298583984375,158.08392990485063,1:22
45.8%,458000,-2839.30517578125,162.97673513640365,1:22
45.9%,459000,-2767.000244140625,152.91117687678596,1:22
46.0%,460000,-2753.868408203125,151.05663762503173,1:22
46.1%,461000,-2750.750732421875,150.61906386502147,1:21
46.2%,462000,-2821.66748046875,160.51235703278107,1:21
46.3%,463000,-2742.351318359375,149.45986618810025,1:21
46.4%,464000,-2753.30029296875,150.99004661323835,1:21
46.5%,465000,-2762.677734375,152.28877561011618,1:21
46.6%,466000,-2746.4521484375,150.02039160408785,1:21
46.7%,467000,-2748.925537109375,150.34895492454677,1:20
46.8%,468000,-2793.13818359375,156.52288815446167,1:20
46.9%,469000,-2785.064208984375,155.4202553886318,1:20
47.0%,470000,-2792.16357421875,156.3998003205461,1:20
47.1%,471000,-2811.116943359375,159.03414551331912,1:20
47.2%,472000,-2750.525634765625,150.59339432366556,1:20
47.3%,473000,-2793.202880859375,156.54687249383204,1:19
47.4%,474000,-2724.37548828125,146.9501918977747,1:19
47.5%,

60.8%,608000,-2805.262939453125,158.23735137450646,0:59
60.9%,609000,-2782.09619140625,155.0218498081043,0:59
61.0%,610000,-2820.15185546875,160.3181332483838,0:58
61.1%,611000,-2856.1162109375,165.3451758825448,0:58
61.2%,612000,-2817.05029296875,159.8847810044586,0:58
61.3%,613000,-2814.886962890625,159.56346915921029,0:58
61.4%,614000,-2862.155517578125,166.16903538658022,0:58
61.5%,615000,-2854.54541015625,165.09219131780222,0:58
61.6%,616000,-2866.7587890625,166.80204151018955,0:58
61.7%,617000,-2848.01806640625,164.19382545931393,0:57
61.8%,618000,-2849.66015625,164.43063889389194,0:57
61.9%,619000,-2863.663818359375,166.3650975731436,0:57
62.0%,620000,-2880.37744140625,168.69399565047186,0:57
62.1%,621000,-2865.8759765625,166.68748181965546,0:57
62.2%,622000,-2821.57861328125,160.52905585245625,0:57
62.3%,623000,-2827.390380859375,161.32482865673222,0:57
62.4%,624000,-2857.204833984375,165.48091124247327,0:56
62.5%,625000,-2835.705078125,162.49124376437953,0:56
62.6%,626000,-286

75.9%,759000,-2819.666015625,160.2911529943724,0:36
76.0%,760000,-2879.91357421875,168.68202901415012,0:36
76.1%,761000,-2861.826416015625,166.17501019361998,0:36
76.2%,762000,-2810.81201171875,159.05727874057558,0:36
76.3%,763000,-2827.392822265625,161.3677927962431,0:35
76.4%,764000,-2810.9912109375,159.09307651608717,0:35
76.5%,765000,-2792.26220703125,156.46993195873338,0:35
76.6%,766000,-2809.5390625,158.8619144659456,0:35
76.7%,767000,-2839.373291015625,163.02977644334334,0:35
76.8%,768000,-2775.71728515625,154.17026107139722,0:35
76.9%,769000,-2818.86865234375,160.1938027907819,0:34
77.0%,770000,-2764.2177734375,152.54904569455388,0:34
77.1%,771000,-2824.904296875,161.01519406968694,0:34
77.2%,772000,-2852.804443359375,164.92574783280338,0:34
77.3%,773000,-2809.61328125,158.89844419787522,0:34
77.4%,774000,-2825.129150390625,161.03432707000795,0:34
77.5%,775000,-2838.146484375,162.86278824659158,0:34
77.6%,776000,-2832.32958984375,162.06989220126067,0:33
77.7%,777000,-2805.77929

91.0%,910000,-2810.580078125,158.96273720696098,0:13
91.1%,911000,-2844.643798828125,163.722343392685,0:13
91.2%,912000,-2827.83203125,161.3785338311208,0:13
91.3%,913000,-2834.48779296875,162.2957773562959,0:13
91.4%,914000,-2871.1201171875,167.40748862355002,0:13
91.5%,915000,-2808.408203125,158.69358151205387,0:12
91.6%,916000,-2795.650390625,156.90794829704285,0:12
91.7%,917000,-2797.25341796875,157.10784048811885,0:12
91.8%,918000,-2813.481689453125,159.37476058131122,0:12
91.9%,919000,-2825.724609375,161.0859044641117,0:12
92.0%,920000,-2815.1201171875,159.61024628099153,0:12
92.1%,921000,-2825.90283203125,161.10813551252735,0:11
92.2%,922000,-2834.526611328125,162.30548003439463,0:11
92.3%,923000,-2855.93359375,165.3003392963833,0:11
92.4%,924000,-2789.577392578125,156.05164439922464,0:11
92.5%,925000,-2811.24560546875,159.0781990763358,0:11
92.6%,926000,-2830.85791015625,161.80284726439612,0:11
92.7%,927000,-2816.005859375,159.72715504096,0:11
92.8%,928000,-2803.33203125,157.96

## Euler's Method (Time Reversed)
We use the final velocities and output ("Euler_forward.pdb") from the forward time evolution. We then invert the velocities as described in the "Reverse Trajectory" section so that we can look at the RMSD of the "time-evolved then time-reversed" structure with the original structure.

In [37]:
compute('E_forward_final.pdb', 'E', True, E_velocities)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Time Remaining"
0.1%,1000,-3499.152587890625,97.38484385358103,--
0.2%,2000,-3387.979248046875,81.83109339423646,2:39
0.3%,3000,-3378.353515625,80.43251192415318,2:45
0.4%,4000,-3415.1337890625,85.57042042227381,2:41
0.5%,5000,-3413.1923828125,85.33760721463123,2:39
0.6%,6000,-3388.12158203125,81.84964763831998,2:37
0.7%,7000,-3348.828125,76.34307141693233,2:36
0.8%,8000,-3402.88623046875,83.8757115143932,2:35
0.9%,9000,-3411.39599609375,85.09090385728943,2:34
1.0%,10000,-3429.128662109375,87.5721169585254,2:34
1.1%,11000,-3368.694091796875,79.13790230990675,2:33
1.2%,12000,-3406.92333984375,84.44336073877433,2:33
1.3%,13000,-3422.188232421875,86.57123464591506,2:33
1.4%,14000,-3393.893310546875,82.64281603952736,2:33
1.5%,15000,-3400.112060546875,83.52022751777086,2:34
1.6%,16000,-3353.202392578125,76.97920963312087,2:33
1.7%,17000,-3350.10791015625,76.51046814749876,2:33
1.8%,18000,-3396.2255859375,82.931249423814

15.6%,156000,-3394.3779296875,82.69516794565477,2:10
15.7%,157000,-3373.6455078125,79.80182933661487,2:09
15.8%,158000,-3392.99267578125,82.50239105185116,2:09
15.9%,159000,-3361.7841796875,78.1373689532631,2:09
16.0%,160000,-3369.635986328125,79.23726113808615,2:09
16.1%,161000,-3377.651611328125,80.3455027327468,2:09
16.2%,162000,-3391.288330078125,82.24456365918039,2:09
16.3%,163000,-3389.7265625,82.02320642059122,2:09
16.4%,164000,-3401.551025390625,83.68425384436615,2:08
16.5%,165000,-3370.98046875,79.43191047854042,2:08
16.6%,166000,-3350.9384765625,76.63508798321939,2:08
16.7%,167000,-3379.44140625,80.59868305327554,2:08
16.8%,168000,-3392.4296875,82.4156882608058,2:08
16.9%,169000,-3366.555908203125,78.8146499290387,2:08
17.0%,170000,-3376.404296875,80.18856617006048,2:08
17.1%,171000,-3378.2763671875,80.44793407565746,2:07
17.2%,172000,-3386.93896484375,81.64672548811839,2:07
17.3%,173000,-3384.802001953125,81.35998581693764,2:07
17.4%,174000,-3396.77587890625,83.0217651971305

31.0%,310000,-3363.434326171875,78.36175615097783,1:46
31.1%,311000,-3377.2626953125,80.28206083577496,1:46
31.2%,312000,-3401.94873046875,83.72732011733068,1:46
31.3%,313000,-3387.96875,81.788214365937,1:46
31.4%,314000,-3382.268798828125,80.9875987337328,1:45
31.5%,315000,-3374.13134765625,79.84706594546991,1:45
31.6%,316000,-3385.67529296875,81.46040002413831,1:45
31.7%,317000,-3364.660888671875,78.54197062997994,1:45
31.8%,318000,-3379.426025390625,80.60152576773605,1:45
31.9%,319000,-3372.177001953125,79.57411429053471,1:45
32.0%,320000,-3391.992919921875,82.34359055363535,1:45
32.1%,321000,-3387.783203125,81.76525136110334,1:44
32.2%,322000,-3376.49072265625,80.18963857132402,1:44
32.3%,323000,-3382.169189453125,80.98465388581862,1:44
32.4%,324000,-3398.96337890625,83.328352802808,1:44
32.5%,325000,-3375.819091796875,80.0906882769593,1:44
32.6%,326000,-3382.8408203125,81.06367964559638,1:44
32.7%,327000,-3361.20458984375,78.05603667965659,1:43
32.8%,328000,-3362.71875,78.26096745

46.4%,464000,-3396.445556640625,82.99856388090501,1:22
46.5%,465000,-3401.83642578125,83.74636800644028,1:22
46.6%,466000,-3423.39697265625,86.75230874815222,1:22
46.7%,467000,-3397.937744140625,83.2011626085033,1:21
46.8%,468000,-3387.481689453125,81.74527575978956,1:21
46.9%,469000,-3399.118896484375,83.35840557155063,1:21
47.0%,470000,-3396.79443359375,83.04237913252977,1:21
47.1%,471000,-3401.4580078125,83.68913071677895,1:21
47.2%,472000,-3390.1552734375,82.1110241685058,1:21
47.3%,473000,-3382.55810546875,81.06179017670348,1:21
47.4%,474000,-3386.321533203125,81.58822855252843,1:20
47.5%,475000,-3385.099365234375,81.40967374214848,1:20
47.6%,476000,-3380.158447265625,80.72600091439733,1:20
47.7%,477000,-3389.952880859375,82.0899336103228,1:20
47.8%,478000,-3404.47705078125,84.1117844814321,1:20
47.9%,479000,-3396.969970703125,83.05900986323583,1:20
48.0%,480000,-3409.815185546875,84.85603095833116,1:19
48.1%,481000,-3412.18212890625,85.18056851849124,1:19
48.2%,482000,-3406.85839

61.8%,618000,-3391.4521484375,82.30209883808159,0:58
61.9%,619000,-3383.30419921875,81.15333579567707,0:58
62.0%,620000,-3416.080810546875,85.71790112937443,0:58
62.1%,621000,-3371.501953125,79.51373639717299,0:58
62.2%,622000,-3377.478515625,80.3434430414311,0:58
62.3%,623000,-3395.0341796875,82.78311336038648,0:58
62.4%,624000,-3392.16845703125,82.38705684929342,0:57
62.5%,625000,-3405.234375,84.20637708177352,0:57
62.6%,626000,-3420.322265625,86.32245457501527,0:57
62.7%,627000,-3380.46337890625,80.7558664384749,0:57
62.8%,628000,-3413.56689453125,85.36360017859045,0:57
62.9%,629000,-3395.682373046875,82.87608884771144,0:57
63.0%,630000,-3405.398193359375,84.23208066761401,0:57
63.1%,631000,-3411.583251953125,85.08960165575513,0:56
63.2%,632000,-3414.332763671875,85.48069618322401,0:56
63.3%,633000,-3420.947021484375,86.38377720282345,0:56
63.4%,634000,-3431.78076171875,87.90796579867953,0:56
63.5%,635000,-3427.337890625,87.29394798633186,0:56
63.6%,636000,-3433.5,88.16787841602942,

77.2%,772000,-3407.15625,84.49393382058365,0:35
77.3%,773000,-3395.7333984375,82.89503460336738,0:34
77.4%,774000,-3393.89111328125,82.64123297099546,0:34
77.5%,775000,-3411.361572265625,85.07221343526767,0:34
77.6%,776000,-3400.51708984375,83.56686846161386,0:34
77.7%,777000,-3422.099609375,86.57481782791467,0:34
77.8%,778000,-3414.82958984375,85.56069221081167,0:34
77.9%,779000,-3405.001220703125,84.18891226119582,0:34
78.0%,780000,-3376.46337890625,80.20952055030527,0:33
78.1%,781000,-3374.09423828125,79.87253121991849,0:33
78.2%,782000,-3412.951904296875,85.29256636156242,0:33
78.3%,783000,-3425.384765625,87.04165282240159,0:33
78.4%,784000,-3426.074951171875,87.1343729760924,0:33
78.5%,785000,-3383.76171875,81.23094019822459,0:33
78.6%,786000,-3393.76513671875,82.6166613642665,0:32
78.7%,787000,-3409.4912109375,84.81025814884437,0:32
78.8%,788000,-3398.4892578125,83.28620573092651,0:32
78.9%,789000,-3390.4541015625,82.17331706412385,0:32
79.0%,790000,-3399.17578125,83.380985575933

92.6%,926000,-3417.62060546875,85.94957384678435,0:11
92.7%,927000,-3419.956298828125,86.28995060338454,0:11
92.8%,928000,-3403.38623046875,83.98332613007766,0:11
92.9%,929000,-3428.44287109375,87.47221341859309,0:10
93.0%,930000,-3403.008544921875,83.91422433754822,0:10
93.1%,931000,-3428.7470703125,87.50094696355914,0:10
93.2%,932000,-3426.2880859375,87.16730250378008,0:10
93.3%,933000,-3411.89111328125,85.16106953996126,0:10
93.4%,934000,-3424.35791015625,86.89103151160057,0:10
93.5%,935000,-3443.38134765625,89.54184572377805,0:10
93.6%,936000,-3397.645751953125,83.16694790152359,0:09
93.7%,937000,-3397.6728515625,83.18114445158382,0:09
93.8%,938000,-3416.357421875,85.77889182345817,0:09
93.9%,939000,-3427.146484375,87.28876471355807,0:09
94.0%,940000,-3431.814453125,87.93576312032027,0:09
94.1%,941000,-3416.84033203125,85.85208746525558,0:09
94.2%,942000,-3439.125732421875,88.94957382594112,0:08
94.3%,943000,-3414.36474609375,85.50641679130678,0:08
94.4%,944000,-3440.41748046875,89

Quantity(value=[Vec3(x=-0.5755617618560791, y=-0.1124480739235878, z=0.5806170701980591), Vec3(x=-0.5093891620635986, y=0.10745643079280853, z=0.026156317442655563), Vec3(x=0.16680675745010376, y=-0.22677931189537048, z=-0.16538596153259277), Vec3(x=0.1661316305398941, y=-0.4016161262989044, z=0.08026359975337982), Vec3(x=-0.2941025495529175, y=0.19291748106479645, z=0.11363465338945389), Vec3(x=0.014446569606661797, y=0.4146529734134674, z=-0.3709012567996979), Vec3(x=0.07397457212209702, y=-0.20590943098068237, z=-0.08012949675321579), Vec3(x=-0.6447877883911133, y=0.07446517795324326, z=-0.06110990419983864), Vec3(x=-0.1973239630460739, y=-0.17953431606292725, z=-0.260564923286438), Vec3(x=0.2326594889163971, y=0.2444538176059723, z=0.38349294662475586), Vec3(x=3.1742286682128906, y=-0.36053621768951416, z=-0.17445841431617737), Vec3(x=-0.3477763831615448, y=1.1505239009857178, z=-1.3664560317993164), Vec3(x=0.060167308896780014, y=0.6347664594650269, z=-1.3886642456054688), Vec3(x=

## Velocity Verlet (Forward in Time):

In [38]:
V_velocities = compute('data/2f4k-processed.pdb','VV', False)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Time Remaining"
0.1%,1000,-2854.21142578125,188.08672134415195,--
0.2%,2000,-2624.794921875,156.09990245608407,2:33
0.3%,3000,-2653.255615234375,160.0731491375121,2:34
0.4%,4000,-2651.556640625,159.83225036478726,2:32
0.5%,5000,-2664.70166015625,161.66477985727548,2:35
0.6%,6000,-2697.6533203125,166.25535517717432,2:34
0.7%,7000,-2576.62646484375,149.37773386910666,2:33
0.8%,8000,-2652.3876953125,159.94296302856642,2:34
0.9%,9000,-2710.674560546875,168.07610527800585,2:33
1.0%,10000,-2711.97802734375,168.25819220365858,2:33
1.1%,11000,-2581.41650390625,150.05469142227895,2:33
1.2%,12000,-2645.599853515625,159.0016670750518,2:32
1.3%,13000,-2683.574462890625,164.29885269417207,2:32
1.4%,14000,-2695.368408203125,165.94486947801514,2:32
1.5%,15000,-2721.109619140625,169.53193254887162,2:33
1.6%,16000,-2619.38427734375,155.3511621072235,2:32
1.7%,17000,-2571.93896484375,148.73233555311862,2:32
1.8%,18000,-2647.282470703

15.4%,154000,-2646.243896484375,159.12397188582258,2:11
15.5%,155000,-2646.31591796875,159.13408309773598,2:11
15.6%,156000,-2650.940185546875,159.78445190846935,2:10
15.7%,157000,-2720.03564453125,169.417645214214,2:10
15.8%,158000,-2672.101318359375,162.72926577815932,2:10
15.9%,159000,-2641.2841796875,158.438315966846,2:10
16.0%,160000,-2642.2275390625,158.5706468783188,2:10
16.1%,161000,-2732.052490234375,171.09845546127423,2:10
16.2%,162000,-2747.5595703125,173.25743751617887,2:09
16.3%,163000,-2691.700927734375,165.47103834195175,2:09
16.4%,164000,-2698.15869140625,166.37837492212083,2:09
16.5%,165000,-2692.263671875,165.55497501862686,2:09
16.6%,166000,-2720.32470703125,169.4627882007365,2:09
16.7%,167000,-2699.050537109375,166.49526665984703,2:09
16.8%,168000,-2716.272216796875,168.90112229451623,2:08
16.9%,169000,-2694.82666015625,165.9068758332496,2:08
17.0%,170000,-2678.425537109375,163.61503517736156,2:08
17.1%,171000,-2690.06884765625,165.24435314152944,2:08
17.2%,172000,-

30.5%,305000,-2705.390625,167.34801290902902,1:47
30.6%,306000,-2755.627197265625,174.35055484857506,1:47
30.7%,307000,-2742.33056640625,172.49114723552916,1:47
30.8%,308000,-2740.57763671875,172.24881859445293,1:47
30.9%,309000,-2790.524658203125,179.22150352104254,1:47
31.0%,310000,-2742.3193359375,172.49502830676866,1:46
31.1%,311000,-2748.16015625,173.3109214014178,1:46
31.2%,312000,-2753.99560546875,174.12281426913148,1:46
31.3%,313000,-2747.400390625,173.20439620923918,1:46
31.4%,314000,-2722.33447265625,169.70916813547498,1:46
31.5%,315000,-2731.060302734375,170.93197793179084,1:46
31.6%,316000,-2717.95068359375,169.10879365031352,1:45
31.7%,317000,-2709.4970703125,167.92736492497662,1:45
31.8%,318000,-2714.45166015625,168.62120854248917,1:45
31.9%,319000,-2742.7548828125,172.56730474748304,1:45
32.0%,320000,-2750.571044921875,173.65185989201316,1:45
32.1%,321000,-2743.72705078125,172.69454600851455,1:45
32.2%,322000,-2767.79931640625,176.05512814585606,1:45
32.3%,323000,-2693.6

45.6%,456000,-2791.178466796875,179.34054006129585,1:23
45.7%,457000,-2773.103759765625,176.81299241657788,1:23
45.8%,458000,-2769.681640625,176.33950172536012,1:23
45.9%,459000,-2710.47119140625,168.08122897293165,1:23
46.0%,460000,-2698.537109375,166.4176792795418,1:22
46.1%,461000,-2758.58154296875,174.78807754185848,1:22
46.2%,462000,-2709.312744140625,167.91893891504878,1:22
46.3%,463000,-2709.60986328125,167.9562176256386,1:22
46.4%,464000,-2741.84326171875,172.45354510233605,1:22
46.5%,465000,-2749.46826171875,173.52230360603176,1:22
46.6%,466000,-2753.5654296875,174.09494885852163,1:21
46.7%,467000,-2752.747802734375,173.9796401893273,1:21
46.8%,468000,-2777.7119140625,177.45605868537376,1:21
46.9%,469000,-2740.09326171875,172.2129697522145,1:21
47.0%,470000,-2716.8017578125,168.95681907125137,1:21
47.1%,471000,-2719.190185546875,169.29426800217968,1:21
47.2%,472000,-2739.7109375,172.15659208578822,1:21
47.3%,473000,-2748.9189453125,173.44556733784046,1:20
47.4%,474000,-2720.92

60.8%,608000,-2716.3837890625,168.92379592123118,0:59
60.9%,609000,-2768.57861328125,176.19568080034932,0:59
61.0%,610000,-2736.43017578125,171.71050720463873,0:59
61.1%,611000,-2692.22705078125,165.55041305769623,0:59
61.2%,612000,-2731.8935546875,171.086642025133,0:59
61.3%,613000,-2700.558837890625,166.71335562791873,0:59
61.4%,614000,-2713.1015625,168.458169505946,0:59
61.5%,615000,-2715.707275390625,168.81950064279098,0:58
61.6%,616000,-2716.99560546875,169.00454943860018,0:58
61.7%,617000,-2760.686279296875,175.0939842578448,0:58
61.8%,618000,-2775.43017578125,177.1497945022996,0:58
61.9%,619000,-2743.14111328125,172.64523257263383,0:58
62.0%,620000,-2675.489501953125,163.21133567948544,0:58
62.1%,621000,-2708.806884765625,167.86021217918807,0:57
62.2%,622000,-2695.90966796875,166.0638549515416,0:57
62.3%,623000,-2704.658935546875,167.28887763935364,0:57
62.4%,624000,-2754.1025390625,174.17757482254132,0:57
62.5%,625000,-2762.447998046875,175.34261112856404,0:57
62.6%,626000,-276

75.9%,759000,-2720.490234375,169.44430204562207,0:36
76.0%,760000,-2734.076416015625,171.34558437467302,0:36
76.1%,761000,-2711.3994140625,168.17927708845562,0:36
76.2%,762000,-2680.013916015625,163.7953177453328,0:36
76.3%,763000,-2757.05908203125,174.5355015109306,0:36
76.4%,764000,-2730.093994140625,170.78025868636288,0:36
76.5%,765000,-2708.06787109375,167.7086461339406,0:35
76.6%,766000,-2679.82666015625,163.76942691482725,0:35
76.7%,767000,-2701.998291015625,166.8686325219829,0:35
76.8%,768000,-2689.990966796875,165.1872435186852,0:35
76.9%,769000,-2723.7666015625,169.8982341804619,0:35
77.0%,770000,-2729.0732421875,170.64038692156072,0:35
77.1%,771000,-2703.203125,167.0277394205597,0:35
77.2%,772000,-2698.685546875,166.40401041899216,0:34
77.3%,773000,-2735.542724609375,171.54925550353482,0:34
77.4%,774000,-2724.0703125,169.94587943659934,0:34
77.5%,775000,-2729.50927734375,170.70993980351054,0:34
77.6%,776000,-2765.193359375,175.6772684117587,0:34
77.7%,777000,-2702.38061523437

91.0%,910000,-2736.033203125,171.65644456316235,0:13
91.1%,911000,-2708.93115234375,167.88678389938474,0:13
91.2%,912000,-2763.73193359375,175.5275407686773,0:13
91.3%,913000,-2762.4853515625,175.35479905403542,0:13
91.4%,914000,-2773.49951171875,176.89262246595655,0:13
91.5%,915000,-2753.94482421875,174.1619313818874,0:13
91.6%,916000,-2748.513671875,173.41108027498427,0:12
91.7%,917000,-2713.75927734375,168.5637754970416,0:12
91.8%,918000,-2679.977294921875,163.85401043670893,0:12
91.9%,919000,-2730.87744140625,170.9478086171098,0:12
92.0%,920000,-2785.78955078125,178.61455845036159,0:12
92.1%,921000,-2782.730224609375,178.1865682127541,0:12
92.2%,922000,-2717.45458984375,169.085166778031,0:11
92.3%,923000,-2681.64111328125,164.09254311775706,0:11
92.4%,924000,-2742.35546875,172.56294705345974,0:11
92.5%,925000,-2787.818603515625,178.89488073620336,0:11
92.6%,926000,-2730.029052734375,170.84485809580966,0:11
92.7%,927000,-2728.95703125,170.69683267695612,0:11
92.8%,928000,-2716.14892

## Velocity Verlet (Time Reversed):

In [39]:
compute('VV_forward_final.pdb', 'VV', True, V_velocities)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Time Remaining"
0.1%,1000,-3456.9423828125,101.73046910709279,--
0.2%,2000,-3334.36865234375,84.64011232614997,3:20
0.3%,3000,-3300.30517578125,79.89191104275253,2:58
0.4%,4000,-3378.2763671875,90.76610241068757,2:50
0.5%,5000,-3380.265625,91.03544535025972,2:50
0.6%,6000,-3361.073486328125,88.3560252599333,2:46
0.7%,7000,-3325.164306640625,83.35218394199785,2:43
0.8%,8000,-3386.1708984375,91.85232573496101,2:42
0.9%,9000,-3373.51904296875,90.09491539764767,2:39
1.0%,10000,-3385.55810546875,91.76871248088918,2:41
1.1%,11000,-3300.57763671875,79.9218446557992,2:39
1.2%,12000,-3350.35205078125,86.86258734475327,2:37
1.3%,13000,-3380.936767578125,91.13065926244433,2:36
1.4%,14000,-3361.5341796875,88.42981668021045,2:35
1.5%,15000,-3366.090576171875,89.06186955825216,2:35
1.6%,16000,-3354.79296875,87.48468221106206,2:35
1.7%,17000,-3309.348876953125,81.1477014334829,2:34
1.8%,18000,-3306.08984375,80.69376929864305,2:34


15.7%,157000,-3369.8720703125,89.58491199674256,2:11
15.8%,158000,-3326.037109375,83.46772241146294,2:11
15.9%,159000,-3329.286376953125,83.92709315271074,2:10
16.0%,160000,-3338.0205078125,85.14505160997723,2:10
16.1%,161000,-3346.25830078125,86.28618017671987,2:10
16.2%,162000,-3343.2958984375,85.87137366575708,2:10
16.3%,163000,-3339.175048828125,85.29871139102495,2:10
16.4%,164000,-3326.535888671875,83.53464535698073,2:10
16.5%,165000,-3351.276123046875,86.98714760262594,2:09
16.6%,166000,-3352.257080078125,87.12041473742404,2:09
16.7%,167000,-3343.96875,85.96777913490116,2:09
16.8%,168000,-3343.94775390625,85.96078299332471,2:09
16.9%,169000,-3349.141357421875,86.68993925244393,2:09
17.0%,170000,-3379.56201171875,90.92722644497438,2:09
17.1%,171000,-3371.951171875,89.87598382858319,2:09
17.2%,172000,-3370.1162109375,89.6120624731767,2:08
17.3%,173000,-3352.21484375,87.11665282188049,2:08
17.4%,174000,-3342.913330078125,85.82441781043192,2:08
17.5%,175000,-3362.29833984375,88.52297

31.0%,310000,-3359.072265625,88.0747412174028,1:47
31.1%,311000,-3359.16796875,88.09458064077835,1:47
31.2%,312000,-3362.3115234375,88.52544763733086,1:47
31.3%,313000,-3342.095703125,85.70952618617343,1:47
31.4%,314000,-3349.8583984375,86.79088114915523,1:47
31.5%,315000,-3330.855224609375,84.14436505315308,1:46
31.6%,316000,-3321.892822265625,82.89590273772359,1:46
31.7%,317000,-3338.802734375,85.25598556290599,1:46
31.8%,318000,-3341.812744140625,85.67313263218207,1:46
31.9%,319000,-3365.563232421875,88.98353319928664,1:46
32.0%,320000,-3340.630615234375,85.50576143497905,1:45
32.1%,321000,-3346.80615234375,86.37027856469662,1:45
32.2%,322000,-3339.36376953125,85.32902800452288,1:45
32.3%,323000,-3334.155517578125,84.60645084204431,1:45
32.4%,324000,-3333.354736328125,84.49043574979542,1:45
32.5%,325000,-3362.87646484375,88.60357972938903,1:45
32.6%,326000,-3347.32763671875,86.43235017116504,1:44
32.7%,327000,-3346.593505859375,86.33346796576944,1:44
32.8%,328000,-3350.4912109375,86

46.3%,463000,-3336.642578125,84.91627267375465,1:23
46.4%,464000,-3363.2021484375,88.61935934798116,1:23
46.5%,465000,-3331.419921875,84.19241884310519,1:23
46.6%,466000,-3329.635986328125,83.94741770999123,1:23
46.7%,467000,-3306.958251953125,80.7815955576788,1:23
46.8%,468000,-3359.082275390625,88.05112285624142,1:22
46.9%,469000,-3345.232421875,86.11988989190154,1:22
47.0%,470000,-3341.19677734375,85.55845378595204,1:22
47.1%,471000,-3334.96923828125,84.6911194751374,1:22
47.2%,472000,-3348.830810546875,86.62077788206653,1:22
47.3%,473000,-3345.912109375,86.21641451674157,1:22
47.4%,474000,-3343.3662109375,85.85986662997684,1:21
47.5%,475000,-3338.12646484375,85.13387650792143,1:21
47.6%,476000,-3355.608642578125,87.56949553321452,1:21
47.7%,477000,-3352.51611328125,87.1412754953363,1:21
47.8%,478000,-3348.154296875,86.53155579916394,1:21
47.9%,479000,-3364.4365234375,88.79666302355366,1:21
48.0%,480000,-3338.5615234375,85.18903708402478,1:21
48.1%,481000,-3353.38330078125,87.255239

61.6%,616000,-3359.273681640625,88.07820524370646,0:59
61.7%,617000,-3345.985595703125,86.2277683523413,0:59
61.8%,618000,-3333.708984375,84.51876927206793,0:59
61.9%,619000,-3337.505859375,85.05119096605378,0:59
62.0%,620000,-3349.26220703125,86.68919027378368,0:59
62.1%,621000,-3349.57470703125,86.73719299700895,0:59
62.2%,622000,-3338.353271484375,85.16494210007961,0:58
62.3%,623000,-3346.943359375,86.3664996269108,0:58
62.4%,624000,-3360.98291015625,88.32069559608433,0:58
62.5%,625000,-3340.77734375,85.50509756753019,0:58
62.6%,626000,-3342.894287109375,85.79877380243943,0:58
62.7%,627000,-3335.27734375,84.73248352387407,0:58
62.8%,628000,-3331.265380859375,84.1717027742523,0:58
62.9%,629000,-3332.52734375,84.34497217840479,0:57
63.0%,630000,-3351.359375,86.97491712154886,0:57
63.1%,631000,-3325.92529296875,83.42631580712056,0:57
63.2%,632000,-3348.03466796875,86.51554638030105,0:57
63.3%,633000,-3349.125,86.66773373739167,0:57
63.4%,634000,-3381.0712890625,91.11960331608445,0:57
6

77.1%,771000,-3336.855224609375,84.95532169754145,0:35
77.2%,772000,-3337.07861328125,84.9842509982937,0:35
77.3%,773000,-3353.873291015625,87.32597533517878,0:35
77.4%,774000,-3341.66162109375,85.61309518366592,0:35
77.5%,775000,-3335.789306640625,84.79923624696907,0:35
77.6%,776000,-3350.50927734375,86.85037388591847,0:35
77.7%,777000,-3349.63427734375,86.72666474015972,0:34
77.8%,778000,-3323.462646484375,83.0743809480133,0:34
77.9%,779000,-3343.129638671875,85.81474066569662,0:34
78.0%,780000,-3364.53271484375,88.80362512064555,0:34
78.1%,781000,-3348.935546875,86.62573135456955,0:34
78.2%,782000,-3353.84716796875,87.31219583005438,0:34
78.3%,783000,-3341.10009765625,85.5342311351898,0:33
78.4%,784000,-3316.396240234375,82.08782285228027,0:33
78.5%,785000,-3339.37451171875,85.29549418723431,0:33
78.6%,786000,-3340.88623046875,85.5059657018864,0:33
78.7%,787000,-3325.23095703125,83.31778199035307,0:33
78.8%,788000,-3324.19189453125,83.17775702537043,0:33
78.9%,789000,-3347.921142578

92.4%,924000,-3326.239501953125,83.47295675096358,0:11
92.5%,925000,-3344.14697265625,85.96708973408889,0:11
92.6%,926000,-3344.70166015625,86.0485752078759,0:11
92.7%,927000,-3350.69873046875,86.88578866097882,0:11
92.8%,928000,-3368.083251953125,89.3105134512137,0:11
92.9%,929000,-3350.94189453125,86.91380727176917,0:11
93.0%,930000,-3374.83642578125,90.24689848783095,0:10
93.1%,931000,-3326.07470703125,83.44637651964574,0:10
93.2%,932000,-3336.3974609375,84.88304525682709,0:10
93.3%,933000,-3327.924560546875,83.70051859686323,0:10
93.4%,934000,-3339.96533203125,85.37723499465551,0:10
93.5%,935000,-3335.45556640625,84.75053561181038,0:10
93.6%,936000,-3332.8876953125,84.39078754349728,0:10
93.7%,937000,-3349.8779296875,86.76339873899664,0:09
93.8%,938000,-3354.5361328125,87.4120142587753,0:09
93.9%,939000,-3356.35400390625,87.66562013536101,0:09
94.0%,940000,-3342.1767578125,85.6820778204994,0:09
94.1%,941000,-3340.38232421875,85.43641281993659,0:09
94.2%,942000,-3333.62451171875,84.

Quantity(value=[Vec3(x=-0.11103136837482452, y=0.060984402894973755, z=-0.13379836082458496), Vec3(x=-0.1400717794895172, y=0.1115594357252121, z=0.12571977078914642), Vec3(x=0.3416896462440491, y=0.2353469729423523, z=-0.11256887763738632), Vec3(x=-0.04018634185194969, y=-0.5281814336776733, z=0.017487069591879845), Vec3(x=-0.06603396683931351, y=-0.24272024631500244, z=0.12175293266773224), Vec3(x=0.3222658336162567, y=0.16798897087574005, z=0.28446879982948303), Vec3(x=-0.23854249715805054, y=-0.08140400052070618, z=-0.16438831388950348), Vec3(x=0.12419528514146805, y=0.3157147169113159, z=-0.06834537535905838), Vec3(x=0.28308144211769104, y=1.8266994953155518, z=-1.166472315788269), Vec3(x=0.6608827114105225, y=1.318169355392456, z=1.6688365936279297), Vec3(x=-1.4343239068984985, y=-0.9980751872062683, z=-1.4283196926116943), Vec3(x=0.3214561939239502, y=-0.4473509192466736, z=0.22732305526733398), Vec3(x=-0.513116717338562, y=-1.5931259393692017, z=0.16314059495925903), Vec3(x=-0.

# References:

[1] http://docs.openmm.org/7.6.0/api-python/generated/openmm.openmm.CustomIntegrator.html
<br>
[2] http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html
<br>
[3] https://scicomp.stackexchange.com/questions/36394/time-reversibility-of-velocity-verlet-algorithm

## Additional Sources:

- https://github.com/choderalab/openmm-tutorials/blob/master/02%20-%20Integrators%20and%20sampling.ipynb
- https://github.com/choderalab/openmmtools/blob/main/openmmtools/integrators.py